In [1]:
import sys
import math

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import scipy.stats as stats

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import roc_curve, classification_report

from multiprocessing import cpu_count
from tqdm import tqdm, tqdm_notebook

from mpengine import mpPandasObj
from util import cprintf

from zig_zag import zig_zag_df
from statsmodels.tsa.stattools import adfuller

from structural_breaks import get_bsadf, get_bsadf0
from sample_weights import mpNumCoEvents, mpSampleW, mpSampleTW, getAvgUniqueness, getIndMatrix
from financial_data_structures import dollar_bar_df
from cross_validation_in_finance import PurgedKFold, cvScore
from labeling import getDailyVol, getTEvents, addVerticalBarrier, getEvents, getBins, getBinsOld, df_returns, df_rolling_autocorr
from fractionally_differentiated_features import fracDiff, fracDiff_FFD, plotMinFFD

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
filepath = '~/Dev/notebook/lopez/data/bitfinex_BTCUSD_trades.csv'
#filepath = '~/Dev/notebook/lopez/data/btcusd_trades.csv'
cols = list(map(str.lower, ['Datetime','Amount','Price','<Unknown>']))
columns = dict(zip(range(len(cols)), cols))

df = pd.read_csv(filepath, header = None).rename(columns = columns).assign(dates = lambda df: (pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M:%S.%f'))).assign(dollar_volume=lambda df: df['amount'] * df['price']).assign(volume=lambda df: df['amount']).drop(['datetime', '<unknown>'], axis = 1).set_index('dates').drop_duplicates()
df = df.iloc[::-1]

In [3]:
dollar_M = 1000000
dollar_df = dollar_bar_df(df, 'dollar_volume', dollar_M)

100%|██████████| 971615/971615 [00:00<00:00, 2973618.27it/s]88636/971615 [00:00<00:00, 2885695.39it/s]


In [4]:
close = dollar_df.price.copy()
close = close[~close.index.duplicated(keep='first')]

logp = np.log(close)

In [5]:
#%matplotlib
#f,ax = plt.subplots()

#logp.plot(ax=ax)

In [6]:
bsadf_ind, bsadf_val = [], []

for i in tqdm(range(1,logp.shape[0])):
    bsadf = get_bsadf0(logP = logp[0:i], minSL = 100, constant ='nc', lags = 2)
    bsadf_ind.append(bsadf['Time'])
    bsadf_val.append(bsadf['gsadf'])
    
bsadf = pd.DataFrame(bsadf_val, index = bsadf_ind).dropna()

#bsadf = get_bsadf(logP = logp, minSL = 2000, constant ='nc', lags = 4)
    

 29%|██▉       | 880/3039 [03:25<08:24,  4.28it/s]     | 115/3039 [00:00<00:02, 1068.60it/s]

KeyboardInterrupt: 

In [7]:
%matplotlib
f,ax = plt.subplots(2)

close.loc[bsadf.index].plot(ax=ax[0])
bsadf.plot(ax=ax[1])


Using matplotlib backend: TkAgg
